In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time


In [2]:
SEASONS = list(range(2016,2023))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR= os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout Error on {url}")
            continue
        else:
            break
    return html


In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
Timeout Error on https://www.basketball-reference.com/leagues/NBA_2017_games-december.html
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
Timeout Error on https://www.basketball-reference.com/leagues/NBA_2017_games-may.html
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18

In [37]:
standings_files = os.listdir(STANDINGS_DIR)

In [38]:
async def scrap_game(standings_files):
    with open(standings_files, 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [39]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
      
    await scrap_game(filepath)
    

Nets vs Bucks, October 19, 2021 | Basketball-Reference.com
Warriors vs Lakers, October 19, 2021 | Basketball-Reference.com
Pacers vs Hornets, October 20, 2021 | Basketball-Reference.com
Bulls vs Pistons, October 20, 2021 | Basketball-Reference.com
Celtics vs Knicks, October 20, 2021 | Basketball-Reference.com
Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Cavaliers vs Grizzlies, October 20, 2021 | Basketball-Reference.com
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
Magic vs Spurs, October 20, 2021 | Basketball-Reference.com
Thunder vs Jazz, October 20, 2021 | Basketball-Reference.com
Kings vs Trail Blazers, October 20, 2021 | Basketball-Reference.com
Nuggets vs Suns, October 20, 2021 | Basketball-Reference.com
Mavericks vs Hawks, October 21, 2021 | Basketball-Reference.com
Bucks vs Heat, October 21, 2021 | Basketball-Reference.com
Clippers vs Warriors, October 21, 2021 | Basketball-

CancelledError: 